# Edge cases for login, init, load

In [ ]:
import lamindb.setup as lnsetup
from lamindb.setup import settings
from lndb.dev._settings_store import (
    current_instance_settings_file,
    current_user_settings_file,
    get_settings_file_name_prefix,
    settings_dir,
)
from lndb.dev import setup_local_test_postgres
from lndb.dev._settings_store import instance_settings_file
import nbproject
import pytest
import os

nbproject.header()

## Log in with in-sufficient information

In [ ]:
with pytest.raises(TypeError):
    lnsetup.login()

If we add an email or handle it looks up the password from the stored env file (`lamin login testuser1@lamin.ai`):

In [ ]:
lnsetup.login("testuser1@lamin.ai")

Now, let's start with a fresh environment without any user profiles stored.

In [ ]:
(
    settings_dir / f"{get_settings_file_name_prefix()}user-testuser1@lamin.ai.env"
).unlink()
(settings_dir / f"{get_settings_file_name_prefix()}user-testuser1.env").unlink()
current_user_settings_file().unlink()

If we try to login with a handle at first login, this will error:

In [ ]:
with pytest.raises(RuntimeError):
    lnsetup.login("testuser1", password="cEvcwMJFX4OwbsYVaMt2Os6GxxGgDUlBGILs2RyS")

If we try to login without password, this will error:

In [ ]:
with pytest.raises(RuntimeError):
    lnsetup.login("testuser1@lamin.ai")

If we try login with a wrong password, this will error:

In [ ]:
result = lnsetup.login("testuser1@lamin.ai", password="hello")
assert result == "could-not-login"

Hence, we need to login once with email and password:

In [ ]:
result = lnsetup.login(
    "testuser1@lamin.ai", password="cEvcwMJFX4OwbsYVaMt2Os6GxxGgDUlBGILs2RyS"
)
assert result is None

And now we can have the convenient login just with the handle.

In [ ]:
result = lnsetup.login("testuser1")
assert result is None

## Configuration with insufficient information

In [ ]:
current_user_settings_file().unlink()

We'd also get an error about not being logged in if we try to init an instance without it being set:

In [ ]:
with pytest.raises(RuntimeError):
    lnsetup.init(storage="mydata")

Let's log in again.

In [ ]:
result = lnsetup.login("testuser1")
assert result is None

Let's now try to init without providing storage (`lamin init`):

In [ ]:
with pytest.raises(TypeError):
    lnsetup.init()

However, if we set a location (`lamin init --storage "mydata"`)

In [ ]:
lnsetup.init(storage="mydata")

## Local instances can not be loaded via the hub

It's not possible to load an instance from the hub if it has a local component, say, local storage.

In [ ]:
lnsetup.init(storage="local_storage_instance")
instance_settings_file("local_storage_instance", "testuser1").unlink()

In [ ]:
assert lnsetup.load("local_storage_instance") == "instance-not-reachable"

It's not possible to load an instance from the hub if it's using a local db.

In [ ]:
pgurl = setup_local_test_postgres("pgtest_remote_storage")

In [ ]:
lnsetup.init(
    storage="s3://lndb-setup-ci-pgtest-remote", db=pgurl, name="pgtest_remote_storage"
)
instance_settings_file("pgtest_remote_storage", "testuser1").unlink()

In [ ]:
assert lnsetup.load("pgtest_remote_storage") == "instance-not-reachable"

In [ ]:
!docker stop pgtest_remote_storage && docker rm pgtest_remote_storage

## Invalid command

In [ ]:
!lamin invalid